In [4]:
import numpy as np
import pandas as pd
import json
import time
import random
import glob
import pyarrow
import pyarrow.parquet as pq

In [ ]:
def processing_data(TARGET_FOLDER):
    
    #Loading snapshot File 
    target_data= pd.concat([pd.read_parquet(file, engine="pyarrow") for file in glob.glob(TARGET_FOLDER)],
                             ignore_index=True)
    
    #CHANGING EMPTY FIELDS TO NaN
    target_data = target_data.replace(np.NaN,"")
    
    # Changing Data Type for better precision
    target_data['Column1'] = target_data['Column1'].astype(str)
    target_data['Column2'] = target_data['Column2'].astype(str)
    target_data['Column3'] = target_data['Column3'].astype(str)
    target_data['Column4'] = target_data['Column4'].astype(str)
    target_data['Column5'] = target_data['Column5'].astype(str)
    target_data['Column6'] = target_data['Column6'].astype(str)
    target_data['Column7'] = target_data['Column7'].astype(str)
    target_data['Column8'] = target_data['Column8'].astype(str)
    
    return target_data



def processing_RAW(raw_data):
    
    #LOADING RAW BATCH-FILE
    raw=pd.read_json(raw_data)
    raw=raw.to_json()
    raw=json.loads(raw)

    #Creating A Dataframe 
    raw=pd.DataFrame.from_dict(raw['Features']['Features'])
    
    #CHANGING EMPTY FIELDS TO NaN
    raw=raw.replace(np.NaN,"")
    
    #SHUFFLING THE DATA
    raw=raw.iloc[np.random.permutation(len(raw))]
    
    
    return raw

In [ ]:
def oneTwoOneTesting(raw,target_data):
    flags=[]
    count=0
    for index,row in raw.iterrows():
        df=target_data.loc[target_data['column1']==row['column1']]
        df=df.loc[df['column2']==row['column2']]
        df=df.loc[df['column3']==row['column3']]
        df=df.loc[df['column4']==row['column4']]
        df=df.loc[df['column5']==row['column5']]
        df=df.loc[df['column6']==row['column6']]
        df=df.loc[df['column7']==row['column7']]
        df=df.loc[df['column8']==row['column8']]
        

        
        count+=1
        if len(df)==0:
            flags.append(index)
        if count==100:
            break
        
            
    return count,flags

In [ ]:
def unitTesting():
    
    #Loading and processing snapshot file
    target= processing_data(TARGET_FOLDER="./target/folder/*")
    
    #Searching Raw Files
    #Adding Specific date for file selection
    year = "2022"
    month = "09"
    day = "08"
    checkDate = year + month + day
    
    raw_files = [file for file in glob.glob("./Raw/folder/*") if checkDate in file]
    #Shuffling files for better testing
    random.shuffle(raw_files)
    raw_files=raw_files[:50]
    file_count=0
    for raw_file in raw_files:
        print("Reading File: [{}]".format(raw_file))
        raw = processing_RAW(raw_file)
        records_scanned,flags=oneTwoOneTesting(raw,target)
        print("TOTAL RECORDS SCANNED: {}\nTOTAL FLAGS RAISED: {}:{}".format(records_scanned,len(flags),flags))
        file_count+=1
    print("Total Batches Read: {}".format(file_count))

In [ ]:
# UNIT TESTING 

# Runtime Completion time

t1 = time.time()
unitTesting()
t2 = time.time()
print("Unit Testing Done in [{} MINUTES {} SECONDS]".format((t2-t1)//60,(t2-t1)%60))